## Overview:

This notebook will assign time blocks based on professor preferences and them optimize classroom assignment.

created on: May 8, 2020

created by: Nanchun (Aslan) Shi

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('class.csv')

In [3]:
df.head(3)

,cancelled,date_of_cancellation,term,course,section,title,mode,units,level,department,...,second_room,seats_offered,reg_count,adj_reg,wait_count,total_tuition_units,classroom_capacity,cap_remaining_seats,classroom_remaining_seats,building
0,False,NaN,20191,ACCT-410,14001,Foundations of Accounting,C,4.0,UG,ACCT,...,NaN,46,44,44.0,0.0,176.0,48.0,2.0,4.0,JFF
1,False,NaN,20191,ACCT-410,14006,Foundations of Accounting,C,4.0,UG,ACCT,...,NaN,47,43,43.0,0.0,172.0,60.0,4.0,17.0,JFF
2,False,NaN,20191,ACCT-373,14058,Introduction to Auditing and Assurance Services,D,0.0,UG,ACCT,...,NaN,105,89,0.0,0.0,0.0,101.0,16.0,12.0,JFF


In [4]:
len(df.course.unique())

167

In [5]:
len(df.first_room.unique())

28

### Full/Half - Future Work

In [6]:
def full_half(text):
    
    text = text.lower()
    if "full semester" in text:
        return 1
    elif "first half" in text or "1st half" in text:
        return 2
    elif "second half" in text or "2nd half" in text:
        return 3
    else:
        return 4

In [7]:
df['full_half'] = df.session_more_info.apply(full_half)

In [8]:
## only 2 instances

df = df[df.full_half != 4]

In [9]:
df.head(3)

,cancelled,date_of_cancellation,term,course,section,title,mode,units,level,department,...,seats_offered,reg_count,adj_reg,wait_count,total_tuition_units,classroom_capacity,cap_remaining_seats,classroom_remaining_seats,building,full_half
0,False,NaN,20191,ACCT-410,14001,Foundations of Accounting,C,4.0,UG,ACCT,...,46,44,44.0,0.0,176.0,48.0,2.0,4.0,JFF,1
1,False,NaN,20191,ACCT-410,14006,Foundations of Accounting,C,4.0,UG,ACCT,...,47,43,43.0,0.0,172.0,60.0,4.0,17.0,JFF,1
2,False,NaN,20191,ACCT-373,14058,Introduction to Auditing and Assurance Services,D,0.0,UG,ACCT,...,105,89,0.0,0.0,0.0,101.0,16.0,12.0,JFF,2


In [10]:
df = df.iloc[:,np.r_[3:6,9:11,18:24,30,35]]

### MW

In [11]:
df = df[df.first_days == 'MW'].reset_index(drop=True).copy()

### Time Blocks

In [12]:
df['time_block'] = list(zip(df.first_days, df.first_begin_time, df.first_end_time))

In [13]:
## get all time blocks

time_blks = df.time_block.value_counts().index.sort_values()

In [14]:
## sorted so that every block is only conflicting with the previous one

time_blks

Index([('MW', '08:00:00', '09:20:00'), ('MW', '08:00:00', '09:50:00'),
       ('MW', '09:30:00', '10:50:00'), ('MW', '10:00:00', '11:50:00'),
       ('MW', '11:00:00', '12:20:00'), ('MW', '12:00:00', '13:50:00'),
       ('MW', '12:30:00', '13:50:00'), ('MW', '14:00:00', '15:20:00'),
       ('MW', '14:00:00', '15:50:00'), ('MW', '15:30:00', '16:50:00'),
       ('MW', '16:00:00', '17:50:00'), ('MW', '17:00:00', '18:20:00'),
       ('MW', '18:00:00', '19:50:00'), ('MW', '18:30:00', '19:50:00')],
      dtype='object')

In [15]:
## get the lookup table for Mon and Wed

lookup = pd.read_excel('Timeslots.xlsx', index_col = 0).loc[['Mon','Wed'],:]

In [16]:
## unique indices for each half-hour time chunk

lookup

,08:00:00,08:30:00,09:00:00,09:30:00,10:00:00,10:30:00,11:00:00,11:30:00,12:00:00,12:30:00,...,17:00:00,17:30:00,18:00:00,18:30:00,19:00:00,19:30:00,20:00:00,20:30:00,21:00:00,21:30:00
Mon,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
Wed,56,57,58,59,60,61,62,63,64,65,...,74,75,76,77,78,79,80,81,82,83


In [17]:
## change columns value type to string, they were originally datatime

lookup.columns = [str(time) for time in lookup.columns]

In [18]:
## so for MW classes, there are two possible class duration windows: 3 half-hour and 4 half-hour
## for each of them I will create a dictionary where keys are the time blocks
## and the values are lists of lists: the first sublist contains Mon indices for that time block, and the second
## sublist contians Wed indices for that time block

time_blks_dict_window_3 = {}
time_blks_dict_window_4 = {}
for tb in time_blks:
    
    mon_start, wed_start = lookup.loc[:,tb[1]]
    duration = pd.to_datetime(tb[2],format='%H:%M:%S') - pd.to_datetime(tb[1],format='%H:%M:%S')
    window = round(duration.seconds/1800)
    
    if window == 3:
        time_blks_dict_window_3[tb] = [list(range(mon_start,mon_start+window)), 
                                       list(range(wed_start,wed_start+window))]
    else:
        time_blks_dict_window_4[tb] = [list(range(mon_start,mon_start+window)), 
                                       list(range(wed_start,wed_start+window))]

In [19]:
## for example, look at the first pair:
## the first sublist [0,1,2] corresponds to the Mon 8:00-8:30, 8:30-9:00, 9:00-9:30 in the lookup table
## the second sublist [56, 57, 58] corresponds to the Wed 8:00-8:30, 8:30-9:00, 9:00-9:30 in the lookup table

time_blks_dict_window_3

{('MW', '08:00:00', '09:20:00'): [[0, 1, 2], [56, 57, 58]],
 ('MW', '09:30:00', '10:50:00'): [[3, 4, 5], [59, 60, 61]],
 ('MW', '11:00:00', '12:20:00'): [[6, 7, 8], [62, 63, 64]],
 ('MW', '12:30:00', '13:50:00'): [[9, 10, 11], [65, 66, 67]],
 ('MW', '14:00:00', '15:20:00'): [[12, 13, 14], [68, 69, 70]],
 ('MW', '15:30:00', '16:50:00'): [[15, 16, 17], [71, 72, 73]],
 ('MW', '17:00:00', '18:20:00'): [[18, 19, 20], [74, 75, 76]],
 ('MW', '18:30:00', '19:50:00'): [[21, 22, 23], [77, 78, 79]]}

In [20]:
time_blks_dict_window_4

{('MW', '08:00:00', '09:50:00'): [[0, 1, 2, 3], [56, 57, 58, 59]],
 ('MW', '10:00:00', '11:50:00'): [[4, 5, 6, 7], [60, 61, 62, 63]],
 ('MW', '12:00:00', '13:50:00'): [[8, 9, 10, 11], [64, 65, 66, 67]],
 ('MW', '14:00:00', '15:50:00'): [[12, 13, 14, 15], [68, 69, 70, 71]],
 ('MW', '16:00:00', '17:50:00'): [[16, 17, 18, 19], [72, 73, 74, 75]],
 ('MW', '18:00:00', '19:50:00'): [[20, 21, 22, 23], [76, 77, 78, 79]]}

### Time assignment

In [21]:
## read in the preference table

pref = pd.read_csv('preference.csv')

In [22]:
pref.head(3)

,name,id,0,1,2,3,4,5,6,7,...,158,159,160,161,162,163,164,165,166,167
0,"Abrams, Scott",8.552623e+09,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,"Adler, Paul, S",8.285354e+09,2,2,2,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,"Akbulut, Rahsan",5.755684e+09,0,3,3,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0


In [23]:
from collections import Counter

In [24]:
window_dict = {3: time_blks_dict_window_3, 4: time_blks_dict_window_4}

In [25]:
## original time blocks should be good enough no need to create new ones
## after all we don't want something like 11:00:00 - 1:50:00
## for reproducability
np.random.seed(1)

tb_counter = Counter()

for tb in time_blks:
    tb_counter[tb] += 0

choice = []
sections = set()

## iterate through all the courses in the dataframe
for idx, cols in df.iterrows():
    
    instructor = cols['first_instructor_uid']
    ## get the preference scores for this instructor
    temp = pref[pref.id == instructor]
    course = cols['course']
    ## create a unique key by combining course and instructor id
    section = course + ' ' + str(instructor)
    ## calculate duration of this course
    duration = pd.to_datetime(cols['first_end_time'],format='%H:%M:%S') - \
                            pd.to_datetime(cols['first_begin_time'],format='%H:%M:%S')
    ## number of half-hour period used by this course
    window = round(duration.seconds/1800)
    ## get the corresponding time block dictionary
    dic = window_dict[window]
    ## assume an instructor could teach different sections but not different courses
    ## the following codes are checking if this course taught by this instructor appears before
    ## in other words, check if this is a course with multiple sections
    ## if it is, will assign the next time block for this course
    if section in sections:
        try:
            pre_idx = df[(df.course == course) & (df.first_instructor_uid == instructor)].loc[:idx-1,:].index[-1]
            pre_tb = choice[pre_idx]
            tb_idx = list(dic.keys()).index(pre_tb)
            cur_tb = list(dic.keys())[tb_idx+1]
            choice.append(cur_tb)
            tb_counter[cur_tb] += 1
        
        ## this exception happends when the last section of the same course is the end of the day
        ## so I will assign a random non-overlapping time blokc for it
        except:
            pre_idxs = list(df[(df.course == course) & (df.first_instructor_uid == instructor)]\
                                                                                        .loc[:idx-1,:].index)
            pre_tbs = set([choice[i] for i in pre_idxs])
            cand = list(set(dic.keys()) - pre_tbs)
            np.random.shuffle(cand)
            choice.append(cand[0])
            tb_counter[cand[0]] += 1
        continue
    sections.add(section)
    
    ## if this course does not have multiple sections or it is the first one appearing
    ## then I will assign time block for it based on highest preference scores
    score_dict = {}
    for tb in dic.keys():
        ## get Mon and Wed scores
        mon_score = temp[[str(num) for num in dic[tb][0]]].sum(axis=1)
        wed_score = temp[[str(num) for num in dic[tb][1]]].sum(axis=1)
        ## add them up
        total = mon_score + wed_score
        ## tracking scores for this course among all time blocks
        score_dict[tb] = total.values[0]
    ## select the time blocks with maximum score
    sorted_tbs = [key for key,value in score_dict.items() if value == max(score_dict.values())]
    
    ## randomly select when tie
    np.random.shuffle(sorted_tbs)
    
    ## The way to avoid out-of-room situation is to set 14 as maximum for each time block
    ## so that adjcent time blocks in total will not have more that 28, which is the total_number of rooms we have
    ## for each of the preferred time blocks, check if the time blocks are below limit
    ## if there is one that is not over the limit, assign it to the current course
    for i in range(len(sorted_tbs)):
        if tb_counter[sorted_tbs[i]] <= 13:
            choice.append(sorted_tbs[i])
            tb_counter[sorted_tbs[i]] += 1
            break
            
    ## this IF happends when all the preferred time blocks by this course are over the limit
    ## so I will assign them a random timeblock that the instructor could word on, so total score should >= window
    if len(choice) < idx + 1:
        l = list(dic.keys())
        np.random.shuffle(l)
        for tb in l:
            if score_dict[tb] > window:
                choice.append(tb)
                tb_counter[tb] += 1
                break
    ## if there is no such block, I will assign them the block with highest scores, even it could have 0 somewhere          
    if len(choice) < idx + 1:  
        l = list(dic.keys())
        scores = pd.Series([score_dict[tb] for tb in l])
        cand_tb = l[scores.idxmax()]
        choice.append(cand_tb)
        tb_counter[cand_tb] += 1

In [26]:
## centered around noon

tb_counter

Counter({('MW', '08:00:00', '09:20:00'): 2,
         ('MW', '08:00:00', '09:50:00'): 11,
         ('MW', '09:30:00', '10:50:00'): 8,
         ('MW', '10:00:00', '11:50:00'): 11,
         ('MW', '11:00:00', '12:20:00'): 8,
         ('MW', '12:00:00', '13:50:00'): 15,
         ('MW', '12:30:00', '13:50:00'): 12,
         ('MW', '14:00:00', '15:20:00'): 11,
         ('MW', '14:00:00', '15:50:00'): 10,
         ('MW', '15:30:00', '16:50:00'): 9,
         ('MW', '16:00:00', '17:50:00'): 14,
         ('MW', '17:00:00', '18:20:00'): 7,
         ('MW', '18:00:00', '19:50:00'): 9,
         ('MW', '18:30:00', '19:50:00'): 8})

In [27]:
## check if all course get assigned

sum(tb_counter.values()) == len(df)

True

In [28]:
df['assigned_time_block'] = choice

In [29]:
df.head(3)

,course,section,title,department,type,first_days,first_begin_time,first_end_time,first_instructor,first_instructor_uid,first_room,seats_offered,classroom_capacity,time_block,assigned_time_block
0,ACCT-473,14135,Financial Statement Auditing,ACCT,ACCT Core,MW,10:00:00,11:50:00,"Layton, Rose, M",7.491812e+09,JFF328,36,36.0,"(MW, 10:00:00, 11:50:00)","(MW, 10:00:00, 11:50:00)"
1,ACCT-473,14136,Financial Statement Auditing,ACCT,ACCT Core,MW,12:00:00,13:50:00,"Layton, Rose, M",7.491812e+09,JFF328,36,36.0,"(MW, 12:00:00, 13:50:00)","(MW, 12:00:00, 13:50:00)"
2,ACCT-530,14206,Ethics for Professional Accountants,ACCT,Elective,MW,09:30:00,10:50:00,"Kling, Gregory",8.348095e+09,JKP104,35,56.0,"(MW, 09:30:00, 10:50:00)","(MW, 12:30:00, 13:50:00)"


### Room Optimization

**NOTE:** optimization below will be conduct for every time block.

**Data:**

- $I$: the set of course in the current time block.
- $J$: the set of avaliable rooms after taking our rooms used by conflicting courses and pre-assigned rooms due to "one course has multiple sections" situation.
- $s_i$: number of seats offered by course $i$.
- $c_j$: room capacity for room $j$.

**Decision variables:** 

- $x_{i,j}$: wether to assign course $i$ to room $j$. (Binary)
- $y_{i,j}$: extra seats for course $i$ if its's assigned to room $j$, with lower bound $L$ and upper bound $U$. (Integer)

**Objective and constraints:** 

$$\begin{aligned}
\text{Maximize:} && \sum_{i \in I,j \in J} \frac{x_{i,j}s_i}{c_j} - \sum_{i \in I,j \in J} y_{i,j}\\
\text{subject to:} \\
\text{(One course in one room)} && \sum_{j \in J} x_{i,j} & = 1 && \text{for each course $i$.}\\
\text{(One room has at most one course)} && \sum_{i \in I} x_{i,j} & \le 1 && \text{for each room $j$.} \\
\text{(Capacity)} && x_{i,j}s_i & \le y_{i,j}+c_j && \text{for each room $j$, for each course $i$.}\\
\text{(Extra seats)} && L \le y_{i,j} & \le U && \text{for each room $j$, for each course $i$.}\\
\end{aligned}$$

In [30]:
from gurobipy import GRB, Model
## for reproducability
np.random.seed(1)

R = set(df.first_room.unique()) ##28 rooms
room = pd.read_csv('room.csv',index_col=0).iloc[:,0]
assignment_dict = {}
courses = set()
previous_rooms = set()

## remeber, time blocks are listed in the way that only adjecent pairs are conflicting with each other
for tb in time_blks:
    
    section = set()
    pre_assigned_rooms = set()
    
    temp = df[df.assigned_time_block == tb].copy()
    temp['id'] = temp.course + temp.section
    
    ## if same course and same instructor, assign same room
    for c,s,ins in zip(temp.course,temp.section,temp.first_instructor_uid):
        c_ins = c+str(ins)
        if c_ins in courses:
            key = [key for key in assignment_dict.keys() if c in key][0]
            r = assignment_dict[key]
            section.add(c+s)
            pre_assigned_rooms.add(r)
            assignment_dict[c+s] = r
    ## keep tracking unique course + instructor keys
    for c,ins in zip(temp.course, temp.first_instructor_uid):
        courses.add(c+str(ins))
    
    ## the following grouobi optimization is maximizing room efficiency and minimize number of extra seats
    ## too few allowance could lead to unfeasible solution
    course = temp[['id','seats_offered']].set_index('id')
    
    mod = Model()
    
    I = set(course.index) - section
    ## if all courses in this time block got assigned above, no need to proceed
    if len(I) == 0:
        continue
    ## rooms in the previous time block could not be used since the time are overlapping
    ## also exclude those assigend above
    J = R - previous_rooms - pre_assigned_rooms
    
    x = mod.addVars(I, J, vtype = GRB.BINARY)
    y = mod.addVars(I, J, lb = 0, ub = 10, vtype = GRB.INTEGER)

    RE = sum(x[i,j]*course.loc[i,:]/room.loc[j] for i in I for j in J)
    extra = sum(y[i,j] for i in I for j in J)
    mod.setObjective(RE - extra, sense = GRB.MAXIMIZE)

    for i in I:
        mod.addConstr(sum(x[i,j] for j in J) == 1)
        for j in J:
            mod.addConstr(x[i,j]*course.loc[i,:] <= y[i,j] + room.loc[j])
    for j in J:
        mod.addConstr(sum(x[i,j] for i in I) <= 1)

    mod.setParam('outputflag',False)
    mod.optimize()
    
    ## rooms used in optimization
    used_rooms = set()
    for i in I:
        for j in J:
            if x[i,j].x:
                used_rooms.add(j)
                assignment_dict[i] = j
                
    ## normally, there would not be overlaps in a particular time block
    ## it could happend when two courses inherit from their previous sections, but in different time blocks
    ## so when that happens will randomly assign a room that could hold the class
    ## all rooms except rooms of previous time block and pre-assigned rooms
    all_rooms = J
    ## avaliable rooms
    ava_rooms = all_rooms - used_rooms
    ## subset of assignment for courses in the current time block
    subset_dict = {k:v for k,v in assignment_dict.items() if k in course.index}
    cum_count = []
    ## iterate through current assigments and check if there are rooms used by multiple courses
    for k,v in subset_dict.items():
        size = course.loc[k,:].values[0]
        if v in cum_count:
            room_sizes = {r:room.loc[r] for r in ava_rooms}
            cand_rooms = [r for r,s in room_sizes.items() if s in range(size-10,size+30)]
            ## if no feasible rooms, extend the range; rare cases
            if len(cand_rooms) == 0:
                cand_rooms = [r for r,s in room_sizes.items() if s in range(size-40,size+60)]
            rand_room = np.random.choice(cand_rooms)
            assignment_dict[k] = rand_room
            ava_rooms.discard(rand_room)
        try:
            ## if re-assignment happened, append the new room so could be used for the next check
            cum_count.append(rand_room)
        except NameError:
            ## if no conflicts, append the original room and move to the next iteration
            cum_count.append(v)
    
    ## after all updates made
    ## get all rooms used for this time block, so that I could exclude them for the next time slot
    new_subset_dict = {k:v for k,v in assignment_dict.items() if k in course.index}
    previous_rooms = set([v for v in new_subset_dict.values()])

Using license file /Users/aslanshi/gurobi.lic
Academic license - for non-commercial use only


In [31]:
## check if all course got assinged a classroom

len(assignment_dict.values()) == len(df)

True

In [32]:
df['id'] = df.course + df.section

In [33]:
df['assigned_room'] = df.id.map(assignment_dict)

In [34]:
df

,course,section,title,department,type,first_days,first_begin_time,first_end_time,first_instructor,first_instructor_uid,first_room,seats_offered,classroom_capacity,time_block,assigned_time_block,id,assigned_room
0,ACCT-473,14135,Financial Statement Auditing,ACCT,ACCT Core,MW,10:00:00,11:50:00,"Layton, Rose, M",7.491812e+09,JFF328,36,36.0,"(MW, 10:00:00, 11:50:00)","(MW, 10:00:00, 11:50:00)",ACCT-47314135,JFF328
1,ACCT-473,14136,Financial Statement Auditing,ACCT,ACCT Core,MW,12:00:00,13:50:00,"Layton, Rose, M",7.491812e+09,JFF328,36,36.0,"(MW, 12:00:00, 13:50:00)","(MW, 12:00:00, 13:50:00)",ACCT-47314136,JFF328
2,ACCT-530,14206,Ethics for Professional Accountants,ACCT,Elective,MW,09:30:00,10:50:00,"Kling, Gregory",8.348095e+09,JKP104,35,56.0,"(MW, 09:30:00, 10:50:00)","(MW, 12:30:00, 13:50:00)",ACCT-53014206,JFF236
3,ACCT-530,14207,Ethics for Professional Accountants,ACCT,Elective,MW,08:00:00,09:20:00,"Kling, Gregory",8.348095e+09,JKP104,33,56.0,"(MW, 08:00:00, 09:20:00)","(MW, 14:00:00, 15:20:00)",ACCT-53014207,JFF236
4,ACCT-530,14208,Ethics for Professional Accountants,ACCT,Elective,MW,15:30:00,16:50:00,"Smith, Harris",5.584314e+09,JKP202,29,54.0,"(MW, 15:30:00, 16:50:00)","(MW, 15:30:00, 16:50:00)",ACCT-53014208,JFF417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,WRIT-340,66743,Advanced Writing,BUCO,WRIT,MW,17:00:00,18:20:00,"Owens, James",6.560221e+09,JFF312,19,20.0,"(MW, 17:00:00, 18:20:00)","(MW, 17:00:00, 18:20:00)",WRIT-34066743,JFF313
131,WRIT-340,66746,Advanced Writing,BUCO,WRIT,MW,17:00:00,18:20:00,"Lee, Lucy V",5.975599e+09,JFF313,19,20.0,"(MW, 17:00:00, 18:20:00)","(MW, 11:00:00, 12:20:00)",WRIT-34066746,JFF313
132,WRIT-340,66747,Advanced Writing,BUCO,WRIT,MW,18:30:00,19:50:00,"Aritz, Jolanta",8.196227e+09,JFF312,19,20.0,"(MW, 18:30:00, 19:50:00)","(MW, 12:30:00, 13:50:00)",WRIT-34066747,JFF312
133,WRIT-340,66748,Advanced Writing,BUCO,WRIT,MW,18:30:00,19:50:00,"Lee, Lucy V",5.975599e+09,JFF313,19,20.0,"(MW, 18:30:00, 19:50:00)","(MW, 12:30:00, 13:50:00)",WRIT-34066748,JKP210
